<a href="https://colab.research.google.com/github/mrariv/model_polarization/blob/main/model_polarization_april22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Новый алгоритм для модели идеологической поляризации

Реализация модели согласно алгоритму, описанному в файле `Базовые модели позиционной поляризации`.

Шаги алгоритма определения принадлежности к партии:
1. Моделирование двух нормальных распределений для идеологической позиции: $\text{position}_{\text{left}} \sim \mathcal{N}(\mu_{\text{left}}, \sigma_{\text{left}}^2)$ и $\text{position}_{\text{right}} \sim \mathcal{N}(\mu_{\text{right}}, \sigma_{\text{right}}^2)$.
2. Добавление когнитивного диссонанса к полученным значениям из пункта 1 (и для правого, и для левого распределений): $\text{position}_{\text{left}} = \text{position}_{\text{left}} + \varepsilon$ и $\text{position}_{\text{right}} = \text{position}_{\text{right}} + \varepsilon$. Когнитивный диссонанс следует стандартному нормальному распределению: $\varepsilon \sim \mathcal{N}(0, 1)$.
3. Сохранение "обрезанных" значений для соответствия с эмпирикой по принципу: $$\text{EmpiricalPosition} = \begin{cases} 0, \text{if} \; \text{position} < 0 \\ \text{position}, \text{if} \; \text{position} \in [0, 10] \\ 10, \text{if} \; \text{position} > 10 \end{cases}$$
4. Моделирование восприятий каждой из партий (республиканцев и демократов) так же с помощью нормального распределения: $\text{perception}_{\text{left}} \sim \mathcal{N}(\mu_{\text{left}}, \sigma_{\text{perception}}^2)$ и $\text{perception}_{\text{right}} \sim \mathcal{N}(\mu_{\text{right}}, \sigma_{\text{perception}}^2)$. Здесь снова учитываем когнитивный диссонанс с помощью добавления $\varepsilon \sim \mathcal{N}(0, 1)$.
5. "Обрезаем" эмпирические восприятие республиканской и демократической партий так же, как делали это с значениями позиций в пункте 3.
6. Присваиваем каждому агенту принадлежность к партии, следуя теории рационального выбора: агент голосует за ту партию, к восприятию которой он ближе по Евклидовому расстоянию.

Рассматриваемые сценарии идеологической поляризации:
1. Поляризация политических элит — симметричный "сдвиг" центров распределений для моделирования восприятия позиций партий `res_pos` и `dem_pos`.
2. Поляризация агентов — симметричный "сдвиг" центров распределений для моделирования идеологической позиции агентов `left_voters` и `right_voters`.
3. Поляризация и политических элит, и агентов — объединение сценариев 1 и 2.

Моделирование поляризации осуществляется с помощью усреднения результатов $n$ числа итераций (параметр `n_iterations`). Такой способ позволяет снизить влияние случайных процессов на результаты, а также улучшает обобщающую способность модели.

> Ещё:
> 1. Попробовать изменение параметров распределения для когнитивного диссонанса (например, снижение / увеличение параметра `scale` метода `np.random.normal`). Возможно `scale` (или стандартное отклонение $\sigma$) $=1$ слишком сильно искажает данные, учитывая, что параметр распределения собственной позиции моделируется со значением стандартного отклонения $\sigma=1.5$.
> 2. Добавление аффективной поляризациии — как делаем, что за алгоритм и тд. Подумать

In [ ]:
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = 8, 5
plt.rcParams['font.size'] = 12
plt.rcParams['savefig.format'] = 'pdf'
sns.set_style('darkgrid')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew, pearsonr
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class IdeologicalPolarizationModel:
    def __init__(self, n_agents: int = 100, n_iterations: int = 100,
                 left_mean: int = 3, right_mean: int = 7, left_std: float = 1.5, right_std: float = 1.5,
                 dem_pos: int = 3, res_pos: int = 7, dem_sd: float = 3.0, res_sd: float = 3.0, random_seed: int = None):
        """
        Parameters
        ----------
        n_agents : int, default = 100
          The number of agents in model.

        n_iterations : int, default = 100
          The number of iterations for averaging metrics.

        left_mean : int, default = 3
          The mean value of left-leaning normal distribution.

        right_mean : int, default = 7
          The mean value of right-leaning normal distribution.

        left_std : float, default = 1.5
          The standard deviation of left-leaning normal distribution.

        right_std : float, default = 1.5
          The standard deviation of right-leaning normal distribution.

        dem_pos : int, default = 3
          The mean value of Democratic party ideological position.

        res_pos : int, default = 7
          The mean value of Republican party ideological position.

        dem_sd : float, default = 3.0
          The standard deviation of perception of Democratic party.

        res_sd : float, default = 3.0
          The standard deviation of perception of Republican party.

        random_seed : int, default = None
          Sets random seed value to ensure reproducibility.
        """

        self.n_agents = n_agents
        self.n_iterations = n_iterations
        self.left_mean = left_mean
        self.right_mean = right_mean
        self.left_std = left_std
        self.right_std = right_std
        self.dem_pos = dem_pos
        self.res_pos = res_pos
        self.dem_sd = dem_sd
        self.res_sd = res_sd
        if random_seed is not None:
            np.random.seed(random_seed)

    def run_single_iteration(self):
        """
        Runs single iteration of a model according to described algorithm.
        """
        n = self.n_agents
        half_n = n // 2

        left = np.random.normal(self.left_mean, self.left_std, half_n) + np.random.normal()
        right = np.random.normal(self.right_mean, self.right_std, half_n) + np.random.normal()
        self_pos = np.concatenate((left, right))

        empir_self = np.clip(self_pos, 0, 10)

        perc_dem = np.random.normal(self.dem_pos, self.dem_sd, n) + np.random.normal()
        perc_res = np.random.normal(self.res_pos, self.res_sd, n) + np.random.normal()

        empir_perc_dem = np.clip(perc_dem, 0, 10)
        empir_perc_res = np.clip(perc_res, 0, 10)

        party = np.where(np.abs(self_pos - perc_dem) < np.abs(self_pos - perc_res), 0, 1)

        df = pd.DataFrame({
            "self": self_pos,
            "empir_self": empir_self,
            "perc_dem": perc_dem,
            "perc_res": perc_res,
            "empir_perc_dem": empir_perc_dem,
            "empir_perc_res": empir_perc_res,
            "party": party
        })

        return df

    def compute_statistics(self, df):
        """
        Computes statistics for results of a model.
        """
        stats = {}

        for col in ['self', 'empir_self']:
            stats[f'{col}_mean'] = df[col].mean()
            stats[f'{col}_std'] = df[col].std()
            stats[f'{col}_skew'] = skew(df[col])

        for col in ['perc_dem', 'perc_res', 'empir_perc_dem', 'empir_perc_res']:
            stats[f'{col}_mean'] = df[col].mean()
            stats[f'{col}_std'] = df[col].std()
            stats[f'{col}_skew'] = skew(df[col])

        for prefix in ['', 'empir_']:
            df[f'{prefix}diff_self_dem'] = df['self'] - df[f'{prefix}perc_dem']
            df[f'{prefix}diff_self_res'] = df['self'] - df[f'{prefix}perc_res']
            df[f'{prefix}diff_dem_res'] = df[f'{prefix}perc_dem'] - df[f'{prefix}perc_res']

        for col in df.columns:
            if col.startswith('diff'):
                stats[f'{col}_mean'] = df[col].mean()
                stats[f'{col}_std'] = df[col].std()
                stats[f'{col}_skew'] = skew(df[col])

        for prefix in ['', 'empir_']:
            for group, label in [(None, 'all'), (0, 'dems'), (1, 'reps')]:
                subset = df if group is None else df[df['party'] == group]
                try:
                    stats[f'{prefix}corr_self_dem_{label}'] = pearsonr(subset['self'], subset[f'{prefix}perc_dem'])[0]
                    stats[f'{prefix}corr_self_res_{label}'] = pearsonr(subset['self'], subset[f'{prefix}perc_res'])[0]
                    stats[f'{prefix}corr_dem_res_{label}'] = pearsonr(subset[f'{prefix}perc_dem'], subset[f'{prefix}perc_res'])[0]
                except Exception:
                    stats[f'{prefix}corr_self_dem_{label}'] = np.nan
                    stats[f'{prefix}corr_self_res_{label}'] = np.nan
                    stats[f'{prefix}corr_dem_res_{label}'] = np.nan

        return stats, df

    def run(self):
        """
        Runs model and computes resulting statistics for given parameters.
        """
        all_stats = []
        for _ in range(self.n_iterations):
            df = self.run_single_iteration()
            stats, _ = self.compute_statistics(df)
            all_stats.append(stats)
        return pd.DataFrame(all_stats).mean().to_frame(name='mean_value')


def visualize_model(n_iterations, n_agents, left_mean, right_mean, dem_pos, res_pos, scenario):
    if scenario == 'voters_only':
        dem_pos, res_pos = 5, 5
    elif scenario == 'parties_only':
        left_mean, right_mean = 5, 5

    model = IdeologicalPolarizationModel(
        n_agents=n_agents,
        n_iterations=n_iterations,
        left_mean=left_mean,
        right_mean=right_mean,
        dem_pos=dem_pos,
        res_pos=res_pos
    )
    _, df = model.compute_statistics(model.run_single_iteration())

    fig = go.Figure()

    for party, color, label in zip([0, 1], ['blue', 'red'], ['Democrats', 'Republicans']):
        subset = df[df['party'] == party]['self']
        fig.add_trace(
            go.Histogram(
                x=subset,
                nbinsx=50,
                marker_color=color,
                name=label,
                opacity=0.6,
                histnorm='probability density'
            )
        )

    fig.update_layout(
        title='Distribution of Voter Positions by Party',
        xaxis_title='Ideological Position (self)',
        yaxis_title='Frequency',
        bargap=0.1,
        hovermode='x unified',
        legend_title='Party',
        template='plotly_white',
    )

    fig.show()

    stats_df = model.run()
    display(stats_df.round(3))

widgets.interact(
    visualize_model,
    n_iterations=widgets.IntSlider(value=100, min=10, max=500, step=10, description='Iterations'),
    n_agents=widgets.IntSlider(value=100, min=50, max=2000, step=50, description='Agents'),
    left_mean=widgets.FloatSlider(value=3.0, min=0, max=10, step=0.5, description='Left mean'),
    right_mean=widgets.FloatSlider(value=7.0, min=0, max=10, step=0.5, description='Right mean'),
    dem_pos=widgets.FloatSlider(value=3.0, min=0, max=10, step=0.5, description='DEM pos'),
    res_pos=widgets.FloatSlider(value=7.0, min=0, max=10, step=0.5, description='RES pos'),
    scenario=widgets.Dropdown(options=['voters_only', 'parties_only', 'both'], description='Scenario')
)

interactive(children=(IntSlider(value=100, description='Iterations', max=500, min=10, step=10), IntSlider(valu…

<function __main__.visualize_model(n_iterations, n_agents, left_mean, right_mean, dem_pos, res_pos, scenario)>

In [ ]:
model =  IdeologicalPolarizationModel(dem_pos=5, res_pos=5)
df = model.run_single_iteration()

In [ ]:
df

,self,empir_self,perc_dem,perc_res,empir_perc_dem,empir_perc_res,party
0,6.448485,6.448485,4.817201,7.075192,4.817201,7.075192,1
1,0.746347,0.746347,4.496717,7.037755,4.496717,7.037755,0
2,3.479133,3.479133,-2.391030,6.306912,0.000000,6.306912,1
3,1.419586,1.419586,2.870589,7.635557,2.870589,7.635557,0
4,4.354694,4.354694,5.949614,1.420536,5.949614,1.420536,0
...,...,...,...,...,...,...,...
95,7.419705,7.419705,1.419656,4.171252,1.419656,4.171252,1
96,5.143572,5.143572,7.013756,8.206519,7.013756,8.206519,0
97,6.713275,6.713275,2.202656,7.378346,2.202656,7.378346,1
98,6.753728,6.753728,2.536129,3.657002,2.536129,3.657002,1


In [ ]:
df['perc_dem'].corr(df['perc_res'])

np.float64(-0.05181053905239792)

Ideas on making correlation between perceptions of parties negatively correlated:
- zero-sum framing (agents must allocate a fixed number of points for each party)
- item response theory (???) — sigmoid for probability, combined with
- make one perception depended on the other (very questionable)

## Method 1: Multivariate Normal Distribution

[Source.](https://towardsdatascience.com/correlated-variables-in-monte-carlo-simulations-19266fb1cf29/) Multivariate normal distribution was used in Monte Carlo simulation to model correlated random normally distributed variables.

Multivariate Normal Distribution:
- models perceptions simultaneously and includes prior knowledge of correlation between data (using covariance matrix, implementation below)
- assumes both variables are normally distributed (we were already doing it in previous model)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew, pearsonr
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import multivariate_normal

class IdeologicalPolarizationModel:
    def __init__(self, n_agents: int = 100, n_iterations: int = 100,
                 left_mean: int = 3, right_mean: int = 7, left_std: float = 1.5, right_std: float = 1.5,
                 dem_pos: int = 3, res_pos: int = 7, dem_sd: float = 3.0, res_sd: float = 3.0, random_seed: int = None):
        """
        Parameters
        ----------
        n_agents : int, default = 100
          The number of agents in model.

        n_iterations : int, default = 100
          The number of iterations for averaging metrics.

        left_mean : int, default = 3
          The mean value of left-leaning normal distribution.

        right_mean : int, default = 7
          The mean value of right-leaning normal distribution.

        left_std : float, default = 1.5
          The standard deviation of left-leaning normal distribution.

        right_std : float, default = 1.5
          The standard deviation of right-leaning normal distribution.

        dem_pos : int, default = 3
          The mean value of Democratic party ideological position.

        res_pos : int, default = 7
          The mean value of Republican party ideological position.

        dem_sd : float, default = 3.0
          The standard deviation of perception of Democratic party.

        res_sd : float, default = 3.0
          The standard deviation of perception of Republican party.

        random_seed : int, default = None
          Sets random seed value to ensure reproducibility.
        """

        self.n_agents = n_agents
        self.n_iterations = n_iterations
        self.left_mean = left_mean
        self.right_mean = right_mean
        self.left_std = left_std
        self.right_std = right_std
        self.dem_pos = dem_pos
        self.res_pos = res_pos
        self.dem_sd = dem_sd
        self.res_sd = res_sd
        if random_seed is not None:
            np.random.seed(random_seed)

    def run_single_iteration(self, corr=-0.7):
        """
        Runs single iteration of a model according to described algorithm.
        """
        n = self.n_agents
        half_n = n // 2

        left = np.random.normal(self.left_mean, self.left_std, half_n) + np.random.normal()
        right = np.random.normal(self.right_mean, self.right_std, half_n) + np.random.normal()
        self_pos = np.concatenate((left, right))

        empir_self = np.clip(self_pos, 0, 10)

        cov_matrix = [[self.dem_sd ** 2, corr * self.dem_sd * self.res_sd],
                      [corr * self.dem_sd * self.res_sd, self.res_sd ** 2]]

        perceptions = multivariate_normal(
        mean=[self.dem_pos, self.res_pos],
        cov=cov_matrix).rvs(self.n_agents)

        perc_dem = perceptions[:, 0]
        perc_res = perceptions[:, 1]

        # perc_dem = np.random.normal(self.dem_pos, self.dem_sd, n) + np.random.normal()
        # perc_res = np.random.normal(self.res_pos, self.res_sd, n) + np.random.normal()

        empir_perc_dem = np.clip(perc_dem, 0, 10)
        empir_perc_res = np.clip(perc_res, 0, 10)

        party = np.where(np.abs(self_pos - perc_dem) < np.abs(self_pos - perc_res), 0, 1)

        df = pd.DataFrame({
            "self": self_pos,
            "empir_self": empir_self,
            "perc_dem": perc_dem,
            "perc_res": perc_res,
            "empir_perc_dem": empir_perc_dem,
            "empir_perc_res": empir_perc_res,
            "party": party
        })

        return df

    def compute_statistics(self, df):
        """
        Computes statistics for results of a model.
        """
        stats = {}

        for col in ['self', 'empir_self']:
            stats[f'{col}_mean'] = df[col].mean()
            stats[f'{col}_std'] = df[col].std()
            stats[f'{col}_skew'] = skew(df[col])

        for col in ['perc_dem', 'perc_res', 'empir_perc_dem', 'empir_perc_res']:
            stats[f'{col}_mean'] = df[col].mean()
            stats[f'{col}_std'] = df[col].std()
            stats[f'{col}_skew'] = skew(df[col])

        for prefix in ['', 'empir_']:
            df[f'{prefix}diff_self_dem'] = df['self'] - df[f'{prefix}perc_dem']
            df[f'{prefix}diff_self_res'] = df['self'] - df[f'{prefix}perc_res']
            df[f'{prefix}diff_dem_res'] = df[f'{prefix}perc_dem'] - df[f'{prefix}perc_res']

        for col in df.columns:
            if col.startswith('diff'):
                stats[f'{col}_mean'] = df[col].mean()
                stats[f'{col}_std'] = df[col].std()
                stats[f'{col}_skew'] = skew(df[col])

        for prefix in ['', 'empir_']:
            for group, label in [(None, 'all'), (0, 'dems'), (1, 'reps')]:
                subset = df if group is None else df[df['party'] == group]
                try:
                    stats[f'{prefix}corr_self_dem_{label}'] = pearsonr(subset['self'], subset[f'{prefix}perc_dem'])[0]
                    stats[f'{prefix}corr_self_res_{label}'] = pearsonr(subset['self'], subset[f'{prefix}perc_res'])[0]
                    stats[f'{prefix}corr_dem_res_{label}'] = pearsonr(subset[f'{prefix}perc_dem'], subset[f'{prefix}perc_res'])[0]
                except Exception:
                    stats[f'{prefix}corr_self_dem_{label}'] = np.nan
                    stats[f'{prefix}corr_self_res_{label}'] = np.nan
                    stats[f'{prefix}corr_dem_res_{label}'] = np.nan

        return stats, df

    def run(self):
        """
        Runs model and computes resulting statistics for given parameters.
        """
        all_stats = []
        for _ in range(self.n_iterations):
            df = self.run_single_iteration()
            stats, _ = self.compute_statistics(df)
            all_stats.append(stats)
        return pd.DataFrame(all_stats).mean().to_frame(name='mean_value')


def visualize_model(n_iterations, n_agents, left_mean, right_mean, dem_pos, res_pos, scenario):
    if scenario == 'voters_only':
        dem_pos, res_pos = 5, 5
    elif scenario == 'parties_only':
        left_mean, right_mean = 5, 5

    model = IdeologicalPolarizationModel(
        n_agents=n_agents,
        n_iterations=n_iterations,
        left_mean=left_mean,
        right_mean=right_mean,
        dem_pos=dem_pos,
        res_pos=res_pos
    )
    _, df = model.compute_statistics(model.run_single_iteration())

    fig = go.Figure()

    for party, color, label in zip([0, 1], ['blue', 'red'], ['Democrats', 'Republicans']):
        subset = df[df['party'] == party]['self']
        fig.add_trace(
            go.Histogram(
                x=subset,
                nbinsx=50,
                marker_color=color,
                name=label,
                opacity=0.6,
                histnorm='probability density'
            )
        )

    fig.update_layout(
        title='Distribution of Voter Positions by Party',
        xaxis_title='Ideological Position (self)',
        yaxis_title='Frequency',
        bargap=0.1,
        hovermode='x unified',
        legend_title='Party',
        template='plotly_white',
    )

    fig.show()

    stats_df = model.run()
    display(stats_df.round(3))

widgets.interact(
    visualize_model,
    n_iterations=widgets.IntSlider(value=100, min=10, max=500, step=10, description='Iterations'),
    n_agents=widgets.IntSlider(value=100, min=50, max=2000, step=50, description='Agents'),
    left_mean=widgets.FloatSlider(value=3.0, min=0, max=10, step=0.5, description='Left mean'),
    right_mean=widgets.FloatSlider(value=7.0, min=0, max=10, step=0.5, description='Right mean'),
    dem_pos=widgets.FloatSlider(value=3.0, min=0, max=10, step=0.5, description='DEM pos'),
    res_pos=widgets.FloatSlider(value=7.0, min=0, max=10, step=0.5, description='RES pos'),
    scenario=widgets.Dropdown(options=['voters_only', 'parties_only', 'both'], description='Scenario')
)

interactive(children=(IntSlider(value=100, description='Iterations', max=500, min=10, step=10), IntSlider(valu…

<function __main__.visualize_model(n_iterations, n_agents, left_mean, right_mean, dem_pos, res_pos, scenario)>

In [ ]:
res = []
for i in range(100):
    model = IdeologicalPolarizationModel(dem_pos=3, res_pos=7)
    df = model.run_single_iteration(corr=-0.7)
    res.append(df['perc_dem'].corr(df['perc_res']))

print(np.mean(res))

-0.7004478808683643


In [ ]:
res = []
for i in range(100):
    model = IdeologicalPolarizationModel(dem_pos=5, res_pos=5)
    df = model.run_single_iteration(corr=-0.7)
    res.append(df['perc_dem'].corr(df['perc_res']))

print(np.mean(res))

-0.6933850868915192


In [ ]:
import plotly.express as px
fig = px.scatter(df, x='perc_res', y='perc_dem', width=700, height=500, trendline='ols', marginal_x='histogram', marginal_y='histogram', title='Bi-Variate Normal Distributions')
fig.show()

Потенциально: корреляция при моделировании априори скорректированных восприятий остаётся равной -0.7 даже при различных средних восприятий (в целом можно было ожидать такой результат). Нужно проверить, изменяется ли корреляция с self party в результате такого моделирования с помощью multivariate normal distributions

### Как изменились метрики:
1. corr_dem_res_all = 0.006 --> corr_dem_res_all = -0.699
2. Разница между восприятиями партий
* diff_dem_res_std = 4.235 --> diff_dem_res_std = 5.534

(отрицательная корреляция увеличивает разброс разницы восприятий, т.к. избиратели сильнее поляризуют партии в своих оценках)
3. Корреляции self с восприятием своей партии:
* corr_self_dem_dems = 0.414 --> corr_self_dem_dems = 0.530
* corr_self_res_reps = 0.424 --> corr_self_res_reps = 0.544

(демократы теперь сильнее отожествляют себя с DEM, а республиканцы — с RES)
4. Корреляции self с восприятием чужой партии
* corr_self_res_dems = -0.132 --> corr_self_res_dems = -0.417
* corr_self_dem_reps = -0.417 --> corr_self_dem_reps = -0.427

(отрицательная корреляция между perc_dem и perc_res => если демократ видит DEM левее, то он видит RES правее (и наоборот))
5. Корреляции между партиями внутри групп
* corr_dem_res_dems = -0.042 --> corr_dem_res_dems = -0.709
* corr_dem_res_reps = -0.053 --> corr_dem_res_reps = -0.709

(общая корреляция (corr = -0.7) наследуется внутри групп, так как восприятие партий связано на индивидуальном уровне)
6. Разницы между self и партиями
* diff_self_dem_mean = 1.852 --> diff_self_dem_mean = 1.932
* diff_self_res_mean = -2.135 --> diff_self_res_mean = -2.072

(Внутри групп изменения сильные: демократы видят DEM ближе, а RES дальше; республиканцы видят RES ближе, а DEM дальше. При этом, общие средние разницы почти не изменились, потому что: у одной группы разница уменьшилась, у другой — увеличилась (у демократов diff_self_dem уменьшился, но diff_self_res стал более отрицательным, а у республиканцев diff_self_res стал ближе к 0, но diff_self_dem вырос))

## Method 2: Copulas with marginal distributions

Copulas with marginal distributions:
- models each party's perception individually
- using copulas to link the distributions
- different copulas can be used for modeling different relationships (gaussian, clayton, gumbel, etc.)
- better usage when we're modeling bounded distributions (for example beta-distributions)

> Note: In case of Gaussian distributions [Gaussian copulas are ultimately the same as multivariate normal distributions](https://stats.stackexchange.com/questions/61956/difference-between-multivariate-standard-normal-distribution-and-gaussian-copula). However, contrary to multivariate normal distributions, **copulas can be used with different distributions and they work with bounded distributions.**

> These differences do not matter if we are implying normally distributed perceptions — however, if we plan on further implementation of different distributions for random variables (bounded distributions, skewed distributions or simply other type of distributions), then copulas are a way to go.

For using copulas we need to use the Kendall's Tau correlation based on rank of values. This decision is based on [source](https://towardsdatascience.com/correlated-variables-in-monte-carlo-simulations-19266fb1cf29/).

In [ ]:
data = pd.read_excel('/content/!база США 01 10 2024-1.xls')
data = data[data['year'] == 2008]
data.year.unique()

array([2008])

In [ ]:
data['my_party_placement'].corr(data['their_party_placement'])

np.float64(-0.7140949650709133)

In [ ]:
from scipy.stats import kendalltau
clean_data = data[['my_party_placement', 'their_party_placement']].dropna()
corr, pvalue = kendalltau(clean_data['my_party_placement'], clean_data['their_party_placement'])
corr, pvalue

(np.float64(-0.579774088615281), np.float64(4.4931061919778115e-171))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew, pearsonr
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import multivariate_normal, norm

class IdeologicalPolarizationModel:
    def __init__(self, n_agents: int = 100, n_iterations: int = 100,
                 left_mean: int = 3, right_mean: int = 7, left_std: float = 1.5, right_std: float = 1.5,
                 dem_pos: int = 3, res_pos: int = 7, dem_sd: float = 3.0, res_sd: float = 3.0, random_seed: int = None):
        """
        Parameters
        ----------
        n_agents : int, default = 100
          The number of agents in model.

        n_iterations : int, default = 100
          The number of iterations for averaging metrics.

        left_mean : int, default = 3
          The mean value of left-leaning normal distribution.

        right_mean : int, default = 7
          The mean value of right-leaning normal distribution.

        left_std : float, default = 1.5
          The standard deviation of left-leaning normal distribution.

        right_std : float, default = 1.5
          The standard deviation of right-leaning normal distribution.

        dem_pos : int, default = 3
          The mean value of Democratic party ideological position.

        res_pos : int, default = 7
          The mean value of Republican party ideological position.

        dem_sd : float, default = 3.0
          The standard deviation of perception of Democratic party.

        res_sd : float, default = 3.0
          The standard deviation of perception of Republican party.

        random_seed : int, default = None
          Sets random seed value to ensure reproducibility.
        """

        self.n_agents = n_agents
        self.n_iterations = n_iterations
        self.left_mean = left_mean
        self.right_mean = right_mean
        self.left_std = left_std
        self.right_std = right_std
        self.dem_pos = dem_pos
        self.res_pos = res_pos
        self.dem_sd = dem_sd
        self.res_sd = res_sd
        if random_seed is not None:
            np.random.seed(random_seed)

    def run_single_iteration(self, corr=corr):
        """
        Runs single iteration of a model according to described algorithm.
        """
        n = self.n_agents
        half_n = n // 2

        left = np.random.normal(self.left_mean, self.left_std, half_n) + np.random.normal()
        right = np.random.normal(self.right_mean, self.right_std, half_n) + np.random.normal()
        self_pos = np.concatenate((left, right))

        empir_self = np.clip(self_pos, 0, 10)

        cov_matrix = [[1, corr],
                      [corr, 1]]

        gaussian_copula = norm.cdf(multivariate_normal(mean=[0, 0], cov=cov_matrix).rvs(n))

        perc_dem = norm.ppf(gaussian_copula[:, 0], loc=self.dem_pos, scale=self.dem_sd)
        perc_res = norm.ppf(gaussian_copula[:, 1], loc=self.res_pos, scale=self.res_sd)

        # perc_dem = np.random.normal(self.dem_pos, self.dem_sd, n) + np.random.normal()
        # perc_res = np.random.normal(self.res_pos, self.res_sd, n) + np.random.normal()

        empir_perc_dem = np.clip(perc_dem, 0, 10)
        empir_perc_res = np.clip(perc_res, 0, 10)

        party = np.where(np.abs(self_pos - perc_dem) < np.abs(self_pos - perc_res), 0, 1)

        df = pd.DataFrame({
            "self": self_pos,
            "empir_self": empir_self,
            "perc_dem": perc_dem,
            "perc_res": perc_res,
            "empir_perc_dem": empir_perc_dem,
            "empir_perc_res": empir_perc_res,
            "party": party
        })

        return df

    def compute_statistics(self, df):
        """
        Computes statistics for results of a model.
        """
        stats = {}

        for col in ['self', 'empir_self']:
            stats[f'{col}_mean'] = df[col].mean()
            stats[f'{col}_std'] = df[col].std()
            stats[f'{col}_skew'] = skew(df[col])

        for col in ['perc_dem', 'perc_res', 'empir_perc_dem', 'empir_perc_res']:
            stats[f'{col}_mean'] = df[col].mean()
            stats[f'{col}_std'] = df[col].std()
            stats[f'{col}_skew'] = skew(df[col])

        for prefix in ['', 'empir_']:
            df[f'{prefix}diff_self_dem'] = df['self'] - df[f'{prefix}perc_dem']
            df[f'{prefix}diff_self_res'] = df['self'] - df[f'{prefix}perc_res']
            df[f'{prefix}diff_dem_res'] = df[f'{prefix}perc_dem'] - df[f'{prefix}perc_res']

        for col in df.columns:
            if col.startswith('diff'):
                stats[f'{col}_mean'] = df[col].mean()
                stats[f'{col}_std'] = df[col].std()
                stats[f'{col}_skew'] = skew(df[col])

        for prefix in ['', 'empir_']:
            for group, label in [(None, 'all'), (0, 'dems'), (1, 'reps')]:
                subset = df if group is None else df[df['party'] == group]
                try:
                    stats[f'{prefix}corr_self_dem_{label}'] = pearsonr(subset['self'], subset[f'{prefix}perc_dem'])[0]
                    stats[f'{prefix}corr_self_res_{label}'] = pearsonr(subset['self'], subset[f'{prefix}perc_res'])[0]
                    stats[f'{prefix}corr_dem_res_{label}'] = pearsonr(subset[f'{prefix}perc_dem'], subset[f'{prefix}perc_res'])[0]
                except Exception:
                    stats[f'{prefix}corr_self_dem_{label}'] = np.nan
                    stats[f'{prefix}corr_self_res_{label}'] = np.nan
                    stats[f'{prefix}corr_dem_res_{label}'] = np.nan

        return stats, df

    def run(self):
        """
        Runs model and computes resulting statistics for given parameters.
        """
        all_stats = []
        for _ in range(self.n_iterations):
            df = self.run_single_iteration()
            stats, _ = self.compute_statistics(df)
            all_stats.append(stats)
        return pd.DataFrame(all_stats).mean().to_frame(name='mean_value')


def visualize_model(n_iterations, n_agents, left_mean, right_mean, dem_pos, res_pos, scenario):
    if scenario == 'voters_only':
        dem_pos, res_pos = 5, 5
    elif scenario == 'parties_only':
        left_mean, right_mean = 5, 5

    model = IdeologicalPolarizationModel(
        n_agents=n_agents,
        n_iterations=n_iterations,
        left_mean=left_mean,
        right_mean=right_mean,
        dem_pos=dem_pos,
        res_pos=res_pos
    )
    _, df = model.compute_statistics(model.run_single_iteration())

    fig = go.Figure()

    for party, color, label in zip([0, 1], ['blue', 'red'], ['Democrats', 'Republicans']):
        subset = df[df['party'] == party]['self']
        fig.add_trace(
            go.Histogram(
                x=subset,
                nbinsx=50,
                marker_color=color,
                name=label,
                opacity=0.6,
                histnorm='probability density'
            )
        )

    fig.update_layout(
        title='Distribution of Voter Positions by Party',
        xaxis_title='Ideological Position (self)',
        yaxis_title='Frequency',
        bargap=0.1,
        hovermode='x unified',
        legend_title='Party',
        template='plotly_white',
    )

    fig.show()

    stats_df = model.run()
    display(stats_df.round(3))

widgets.interact(
    visualize_model,
    n_iterations=widgets.IntSlider(value=100, min=10, max=500, step=10, description='Iterations'),
    n_agents=widgets.IntSlider(value=100, min=50, max=2000, step=50, description='Agents'),
    left_mean=widgets.FloatSlider(value=3.0, min=0, max=10, step=0.5, description='Left mean'),
    right_mean=widgets.FloatSlider(value=7.0, min=0, max=10, step=0.5, description='Right mean'),
    dem_pos=widgets.FloatSlider(value=3.0, min=0, max=10, step=0.5, description='DEM pos'),
    res_pos=widgets.FloatSlider(value=7.0, min=0, max=10, step=0.5, description='RES pos'),
    scenario=widgets.Dropdown(options=['voters_only', 'parties_only', 'both'], description='Scenario')
)

interactive(children=(IntSlider(value=100, description='Iterations', max=500, min=10, step=10), IntSlider(valu…

<function __main__.visualize_model(n_iterations, n_agents, left_mean, right_mean, dem_pos, res_pos, scenario)>

In [ ]:
res = []
for i in range(100):
    model = IdeologicalPolarizationModel(dem_pos=3, res_pos=7)
    df = model.run_single_iteration()
    res.append(df['perc_dem'].corr(df['perc_res']))

print(np.mean(res))

-0.5718192310799854


In [ ]:
fig = px.scatter(df, x='perc_res', y='perc_dem', width=700, height=500, trendline='ols', marginal_x='histogram', marginal_y='histogram', title='Two Dependent Copula Distributions')
fig.show()

In [ ]:
df

,self,empir_self,perc_dem,perc_res,empir_perc_dem,empir_perc_res,party
0,5.843267,5.843267,5.232758,6.666280,5.232758,6.666280,0
1,1.972038,1.972038,3.860535,0.944002,3.860535,0.944002,1
2,0.657531,0.657531,0.472803,9.904901,0.472803,9.904901,0
3,4.355450,4.355450,-0.909163,8.293945,0.000000,8.293945,1
4,4.004395,4.004395,-0.909119,10.072887,0.000000,10.000000,0
...,...,...,...,...,...,...,...
95,8.527908,8.527908,2.555494,13.469885,2.555494,10.000000,1
96,6.001872,6.001872,5.284171,6.135274,5.284171,6.135274,1
97,5.399964,5.399964,8.044661,2.693298,8.044661,2.693298,0
98,8.830401,8.830401,0.266740,5.928999,0.266740,5.928999,1


#### Example on modelling random variables with some non-normal distributions (code taken from [here](https://www.youtube.com/watch?v=WFEzkoK7tsE)).

In [ ]:
from scipy.stats import gamma
from scipy.stats import beta
import plotly.figure_factory as ff

np.random.seed(seed=5)
mean = [0,0]
rho = 0.8
cov = [[1,rho],[rho,1]]

norm_1,norm_2 = np.random.multivariate_normal(mean, cov, 1000).T
unif_1 = norm.cdf(norm_1)
unif_2 = norm.cdf(norm_2)

website_time = pd.DataFrame(gamma.ppf(unif_1, a=2, scale=5))
website_spend =  pd.DataFrame(beta.ppf(unif_2,a=0.5, b=0.5, loc=5, scale=100))
join_time_spend = pd.concat([website_time, website_spend], axis=1)
join_time_spend.columns = ['Time', 'Cash']
gamma_dist  = ff.create_distplot([website_time.values.reshape(-1)], group_labels = [' '])
gamma_dist.update_layout(showlegend=False, title_text='Time Spent on Website', width=1000, height=500)
gamma_dist.show()

In [ ]:
t_dist  = ff.create_distplot([website_spend.values.reshape(-1)], group_labels = [' '])
t_dist.update_layout(showlegend=False, title_text='Dollars Spent on Website', width=1000, height=500)
t_dist.show()

In [ ]:
fig = px.scatter(join_time_spend, x = 'Time', y='Cash', width=1000, height=500,  range_y=[0,110], trendline='ols', trendline_color_override='DeepPink',  marginal_x='histogram', marginal_y='histogram')
fig.show()

In [ ]:
print("Corrlation between time and $$ is: " + str(round(join_time_spend.corr().values[0][1],4)))

Corrlation between time and $$ is: 0.7231


For further reading on copulas: Section 14.3 Introduction to Copulas [here](https://ewfrees.github.io/Loss-Data-Analytics-Spanish/C-DependenceModel.html).

## Model 3: Mixture models

Using Gaussian Mixture Models is unnecessarily complicated for our goal, but it can provide insights if we're planning on further clustering data.

It works in a way 'backwards' in our case: the algorithm models two separate clusters (`perc_res` and `perc_dem`) based on the normally distributed variables for each clusters.

The benefits of using Gaussian Mixture Model include the ability to further advance model and cluster agents using some other input data. For example, the algorithm allows for clustering agents based on the type of perceptions they have: we can label agents on whether they think both parties are left-leaning / right-leaning or parties are negatively correlated (Republican party is right-leaning, Democratic party is left-leaning).

Again, the usage of Gaussian Mixture Model makes sense if we're implying different subgroups of agents.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew, pearsonr
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import multivariate_normal, norm
from sklearn.mixture import GaussianMixture

class IdeologicalPolarizationModel:
    def __init__(self, n_agents: int = 100, n_iterations: int = 100,
                 left_mean: int = 3, right_mean: int = 7, left_std: float = 1.5, right_std: float = 1.5,
                 dem_pos: int = 3, res_pos: int = 7, dem_sd: float = 3.0, res_sd: float = 3.0, random_seed: int = None):
        """
        Parameters
        ----------
        n_agents : int, default = 100
          The number of agents in model.

        n_iterations : int, default = 100
          The number of iterations for averaging metrics.

        left_mean : int, default = 3
          The mean value of left-leaning normal distribution.

        right_mean : int, default = 7
          The mean value of right-leaning normal distribution.

        left_std : float, default = 1.5
          The standard deviation of left-leaning normal distribution.

        right_std : float, default = 1.5
          The standard deviation of right-leaning normal distribution.

        dem_pos : int, default = 3
          The mean value of Democratic party ideological position.

        res_pos : int, default = 7
          The mean value of Republican party ideological position.

        dem_sd : float, default = 3.0
          The standard deviation of perception of Democratic party.

        res_sd : float, default = 3.0
          The standard deviation of perception of Republican party.

        random_seed : int, default = None
          Sets random seed value to ensure reproducibility.
        """

        self.n_agents = n_agents
        self.n_iterations = n_iterations
        self.left_mean = left_mean
        self.right_mean = right_mean
        self.left_std = left_std
        self.right_std = right_std
        self.dem_pos = dem_pos
        self.res_pos = res_pos
        self.dem_sd = dem_sd
        self.res_sd = res_sd
        if random_seed is not None:
            np.random.seed(random_seed)

    def run_single_iteration(self, corr=-0.7):
        """
        Runs single iteration of a model according to described algorithm.
        """
        n = self.n_agents
        half_n = n // 2

        left = np.random.normal(self.left_mean, self.left_std, half_n) + np.random.normal()
        right = np.random.normal(self.right_mean, self.right_std, half_n) + np.random.normal()
        self_pos = np.concatenate((left, right))

        empir_self = np.clip(self_pos, 0, 10)

        means = [
            [self.dem_pos, self.res_pos],
            [self.res_pos, self.dem_pos]
        ]

        covs = [
            [[self.dem_sd**2, corr*self.dem_sd*self.res_sd],
            [corr*self.dem_sd*self.res_sd, self.res_sd**2]],

            [[self.dem_sd**2, corr*self.dem_sd*self.res_sd],
            [corr*self.dem_sd*self.res_sd, self.res_sd**2]]
        ]

        gmm = GaussianMixture(n_components=2, covariance_type='full')
        gmm.means_ = np.array(means)
        gmm.covariances_ = np.array(covs)

        gmm.weights_ = np.array([0.5, 0.5])
        data, labels = gmm.sample(self.n_agents)

        perc_dem = data[:, 0]
        perc_res = data[:, 1]

        empir_perc_dem = np.clip(perc_dem, 0, 10)
        empir_perc_res = np.clip(perc_res, 0, 10)

        party = np.where(np.abs(self_pos - perc_dem) < np.abs(self_pos - perc_res), 0, 1)

        df = pd.DataFrame({
            "self": self_pos,
            "empir_self": empir_self,
            "perc_dem": perc_dem,
            "perc_res": perc_res,
            "empir_perc_dem": empir_perc_dem,
            "empir_perc_res": empir_perc_res,
            "party": party
        })

        return df

    def compute_statistics(self, df):
        """
        Computes statistics for results of a model.
        """
        stats = {}

        for col in ['self', 'empir_self']:
            stats[f'{col}_mean'] = df[col].mean()
            stats[f'{col}_std'] = df[col].std()
            stats[f'{col}_skew'] = skew(df[col])

        for col in ['perc_dem', 'perc_res', 'empir_perc_dem', 'empir_perc_res']:
            stats[f'{col}_mean'] = df[col].mean()
            stats[f'{col}_std'] = df[col].std()
            stats[f'{col}_skew'] = skew(df[col])

        for prefix in ['', 'empir_']:
            df[f'{prefix}diff_self_dem'] = df['self'] - df[f'{prefix}perc_dem']
            df[f'{prefix}diff_self_res'] = df['self'] - df[f'{prefix}perc_res']
            df[f'{prefix}diff_dem_res'] = df[f'{prefix}perc_dem'] - df[f'{prefix}perc_res']

        for col in df.columns:
            if col.startswith('diff'):
                stats[f'{col}_mean'] = df[col].mean()
                stats[f'{col}_std'] = df[col].std()
                stats[f'{col}_skew'] = skew(df[col])

        for prefix in ['', 'empir_']:
            for group, label in [(None, 'all'), (0, 'dems'), (1, 'reps')]:
                subset = df if group is None else df[df['party'] == group]
                try:
                    stats[f'{prefix}corr_self_dem_{label}'] = pearsonr(subset['self'], subset[f'{prefix}perc_dem'])[0]
                    stats[f'{prefix}corr_self_res_{label}'] = pearsonr(subset['self'], subset[f'{prefix}perc_res'])[0]
                    stats[f'{prefix}corr_dem_res_{label}'] = pearsonr(subset[f'{prefix}perc_dem'], subset[f'{prefix}perc_res'])[0]
                except Exception:
                    stats[f'{prefix}corr_self_dem_{label}'] = np.nan
                    stats[f'{prefix}corr_self_res_{label}'] = np.nan
                    stats[f'{prefix}corr_dem_res_{label}'] = np.nan

        return stats, df

    def run(self):
        """
        Runs model and computes resulting statistics for given parameters.
        """
        all_stats = []
        for _ in range(self.n_iterations):
            df = self.run_single_iteration()
            stats, _ = self.compute_statistics(df)
            all_stats.append(stats)
        return pd.DataFrame(all_stats).mean().to_frame(name='mean_value')


def visualize_model(n_iterations, n_agents, left_mean, right_mean, dem_pos, res_pos, scenario):
    if scenario == 'voters_only':
        dem_pos, res_pos = 5, 5
    elif scenario == 'parties_only':
        left_mean, right_mean = 5, 5

    model = IdeologicalPolarizationModel(
        n_agents=n_agents,
        n_iterations=n_iterations,
        left_mean=left_mean,
        right_mean=right_mean,
        dem_pos=dem_pos,
        res_pos=res_pos
    )
    _, df = model.compute_statistics(model.run_single_iteration())

    fig = go.Figure()

    for party, color, label in zip([0, 1], ['blue', 'red'], ['Democrats', 'Republicans']):
        subset = df[df['party'] == party]['self']
        fig.add_trace(
            go.Histogram(
                x=subset,
                nbinsx=50,
                marker_color=color,
                name=label,
                opacity=0.6,
                histnorm='probability density'
            )
        )

    fig.update_layout(
        title='Distribution of Voter Positions by Party',
        xaxis_title='Ideological Position (self)',
        yaxis_title='Frequency',
        bargap=0.1,
        hovermode='x unified',
        legend_title='Party',
        template='plotly_white',
    )

    fig.show()

    stats_df = model.run()
    display(stats_df.round(3))

widgets.interact(
    visualize_model,
    n_iterations=widgets.IntSlider(value=100, min=10, max=500, step=10, description='Iterations'),
    n_agents=widgets.IntSlider(value=100, min=50, max=2000, step=50, description='Agents'),
    left_mean=widgets.FloatSlider(value=3.0, min=0, max=10, step=0.5, description='Left mean'),
    right_mean=widgets.FloatSlider(value=7.0, min=0, max=10, step=0.5, description='Right mean'),
    dem_pos=widgets.FloatSlider(value=3.0, min=0, max=10, step=0.5, description='DEM pos'),
    res_pos=widgets.FloatSlider(value=7.0, min=0, max=10, step=0.5, description='RES pos'),
    scenario=widgets.Dropdown(options=['voters_only', 'parties_only', 'both'], description='Scenario')
)

interactive(children=(IntSlider(value=100, description='Iterations', max=500, min=10, step=10), IntSlider(valu…

<function __main__.visualize_model(n_iterations, n_agents, left_mean, right_mean, dem_pos, res_pos, scenario)>

In [ ]:
res = []
for i in range(100):
    model = IdeologicalPolarizationModel(dem_pos=3, res_pos=7)
    df = model.run_single_iteration(corr=-0.58)
    res.append(df['perc_dem'].corr(df['perc_res']))

print(np.mean(res))

-0.7084024517330901


In [ ]:
import plotly.express as px
fig = px.scatter(df, x='perc_res', y='perc_dem', width=700, height=500, trendline='ols', marginal_x='histogram', marginal_y='histogram', title='Dependent Variables Using Gaussian Mixture Model')
fig.show()

#### Past

Source: https://mlisi.xyz/post/simulating-correlated-variables-with-the-cholesky-factorization/. This approach is similar to previous models in a way that it also uses covariance matrix to ensure correlation between random variables. The main difference is the way it computes perceptions:
1. Using Cholesky decomposition on covariance matrix (taking upper triangular square root of covariance matrix denoted as $\Sigma$): $$\Sigma = LL^T$$
2. Generating uncorrelated perceptions `uncorrelated` $\sim \mathcal{N}(0, 1)$
3. "Correlating" data by multiplying `uncorrelated` with $L^T$
4. Modelling perceptions this way:
`perc_dem = self.dem_pos + correlated[:, 0] * self.dem_sd` and
`perc_res = self.res_pos + correlated[:, 1] * self.res_sd`